# Custom Markdown Parser

Inputs markdown, outputs XML based on a custom schema using regex.

Next steps:
- Code batch processing (i.e. open all files in one directory, output to another).
- Automate transfer of process files to new directory to simplify workflow.

In [196]:
# Notes on Version 2 (From General Hanley):

# Advocates transcending the "line-by-line approach":
# If you give re.sub the flags=re.MULTILINE option, then ^ and $ will match the
# beginning and end of lines, rather than the whole string. That way you
# can do it for the whole file at once. This should in principle be faster too.

In [7]:
import re, os

### File Paths

In [8]:
input_path = r"/Users/Enkidu/Box Sync/Notes/Primary Sources/transcription_markdown_drafting_stage1/"

os.listdir(input_path)

['.DS_Store',
 '.Ulysses-Group.plist',
 'archive_docs_now_at_xml_stage_do_not_use',
 'document_conversion_backlog',
 'ser560.txt',
 'ser706.txt']

In [9]:
output_path = r"/Users/Enkidu/Box Sync/Notes/Primary Sources/xml_notes_stage2/parser_depository/"
os.listdir(output_path)

[]

In [10]:
input_file = 'ser706.txt'
ret = re.match(r'[^0-9]*([0-9]+)\.txt', input_file)
doc_serial, = ret.groups()
print(doc_serial)

doc_serial_xml = '<document serial = "' + doc_serial + '">'
print (doc_serial_xml)

706
<document serial = "706">


In [13]:
os.chdir(input_path)
with open(input_file) as fin:
    text = fin.read()

Previous code, when `<lb>` was wrapped instead of a milestone:

```python

conv = re.sub(r'^ *\- (.*)$', r'<lb>\1</lb>', conv, flags=re.MULTILINE)

```

In [14]:
# General Hanley on F-strings: I made a few other changes too, the major one is useing f-strings,
# which is a great new feature. (You need to have python version >=3.6)


conv = text
conv = re.sub(r'^ *\- ', r'\t<lb/>', conv, flags=re.MULTILINE)
conv = re.sub(r'\----', r'</div>\n<div>', conv, flags=re.MULTILINE)
conv = re.sub(r'(\*[^\*\n]+\*)', r'<flag>\1</flag>', conv)
conv = re.sub(r'\*', r'', conv)
conv = re.sub(r'^> (.*)$', r'<!-- \1 -->', conv, flags=re.MULTILINE)
# print(conv)

In [15]:
body = '\n'.join([f'\t{s}' for s in conv.split('\n')])
final = f"""
<?xml-model href="/Users/Enkidu/Documents/digital_humanities/xml_development/schemas/persian_documents_schema_basic.rnc" type="application/relax-ng-compact-syntax"?>
<document>
{body}
</document>
""".strip()

final = re.sub(r'</div>', '', final, count=1)
final = re.sub (r'<document>', doc_serial_xml, final)

print(final)



<?xml-model href="../../../../../Documents/digital_humanities/xml_development/schemas/schema_coding_primary_texts_2.0.rnc" type="application/relax-ng-compact-syntax"?>
<document serial = "706">
	
	
	
	<div> 
		<lb/>جهان پناها 
	
	</div>
	<div> 
	
		<lb/>تصدق سر مبارک جناب عالیحضرتم و سیدم هزاران هزار بار شوم مبارک نماهً
		<lb/>همایون <flag>عنبر</flag> شمایم  باینغلام رضاجوی جانسپار محقر سراپا عیب و نقصان از وجه
		<lb/>تعفن ناک جمع می شده گی باشد و آب باران بخندقهای جمع شده بدبوی میشده گی
		<lb/>باشد بدرون قورغان و بقشلاغهای دور نزدیک ها را بطرف کنانیده پاکیزه میکنید
		<lb/>گفته مرحمت و مهربانی عالیملایم شده بوده است که اینغلام رضاجوی مبارکنامهً
		<lb/>عالی سیدم را بدو دست ادم گرفته بوسیده بچشمان خود مالیده کحل البصر و تاج
		<lb/>سر خود کرده فوراً باقسقالان و بفقرایان درون قورغان و بقشلاقهای 
		<lb/>دور نزدیک آدم فرموده جمع کنانیده آورده سپارش مرحمت جناب عالی حضرتم را 
		<lb/>گفته بودم که جمیع فقرا غلامشان از اینوجه خورسند و سرافراز بلا نهایه
		<lb/>گردیده دعای بلا نهایه نمودند که اینغل

In [16]:
output_file = 'parser_output.xml'
with open(output_file, 'w+') as fout:
    fout.write(final)